In [ ]:
#| default_exp storage

# Sqlite

> Essa seção carrega as informações para o banco e permite usuários de realizem queries nas informações.

In [ ]:
#| exporti
import sqlite3
import pandas as pd

In [ ]:
#| hide
import tempfile

## Wrapper

A seguinte seção adiciona uma cada de abstração para a execução de queries SQLite. A presente aplicação requer o carregamento inicial do conjunto de dados abertos da UFRN, que é então disponibilizado para leitura de diversos usuários.

In [ ]:
#| export
class ReadOnlyInMemorySQLite:
    def __init__(self):
        self.connection = sqlite3.connect('file:storage.db?mode=memory&cache=shared', uri=False)
        self.load_csv_as_table('turma_matricula_docente_filtrados.csv', 'data')

    def load_csv_as_table(self, csv_path, table_name):
        df = pd.read_csv(csv_path)
        df.to_sql(table_name, self.connection, if_exists='replace', index=False)

    def execute_query(self, query):
        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(query)
            return cursor.fetchall()

Precisa-se checar se a máquina que vai receber a aplicação tem memória suficiente para roda-la.

In [ ]:
def get_memory_usage_of_db(connection):
    with tempfile.NamedTemporaryFile(delete=True) as temp_file:
        file_connection = sqlite3.connect(temp_file.name)
        connection.backup(file_connection)
        file_connection.close()

        size_in_bytes = os.path.getsize(temp_file.name)
        size_in_mb = size_in_bytes / (1024 * 1024)

        return size_in_mb

db = ReadOnlyInMemorySQLite()
memory_usage = get_memory_usage_of_db(db.connection)
print(f"Memory usage of the database: {memory_usage} bytes")

In [ ]:
db.execute_query(f'SELECT unidade_responsavel FROM data LIMIT 1')

In [ ]:
unidade_responsavel = 'DEPARTAMENTO DE ARQUITETURA'
nome_componente = 'ATELIER INTEGRADO DE ARQUITETURA E URBANISMO'
docente = 'BIANCA CARLA DANTAS DE ARAUJO'

In [ ]:
q = f"""SELECT descricao, COUNT(*) as contagem
FROM (
    SELECT DISTINCT discente, descricao
    FROM data
    WHERE unidade_responsavel = '{unidade_responsavel}'
    AND nome_componente = '{nome_componente}'
    AND nome_docente = '{docente}'
)
GROUP BY descricao;"""

In [ ]:
db.execute_query(q)

In [ ]:
db.execute_query("SELECT DISTINCT(nome_componente) from data WHERE unidade_responsavel = 'DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA' ORDER BY nome_componente")